In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [ ]:
human_labels = pd.read_csv(
    "../dataset/saxony-labeled-data/human-labels.csv", dtype={"post_id": str}
)
human_labels = (
    human_labels.sort_values(by="timestamp")
    .rename(
        columns={
            "is_saxony_election": "is_saxony",
            "is_saxony_election_comment": "is_saxony_comment",
        }
    )  # The column name was wrong while saving, but the UI correctly displayed that it does not HAVE to be politics in saxony
    .groupby(["post_id", "classification_by"])
    .last()
    .reset_index()
)
print(len(human_labels))
# join num raters per post
nraters_by_post = (
    human_labels.groupby("post_id", as_index=False)["classification_by"]
    .agg(["nunique"])
    .rename(columns={"nunique": "nraters"})
)
human_labels = pd.merge(human_labels, nraters_by_post, on="post_id", how="left")
human_labels.head()

In [4]:
questions = [
    "is_political",
    "is_saxony",
    "is_intolerant",
    "is_hedonic_entertainment",
    "is_eudaimonic_entertainment",
]
comment_cols = [f"{col}_comment" for col in questions]


# Krippendorff's alpha

In [ ]:
import matplotlib.pyplot as plt

from bench_lib.evaluation import krippendorf_alpha, plot_krippendorf_alphas


alphas = [
    krippendorf_alpha(human_labels["post_id"], human_labels[q]) for q in questions
]
plot_krippendorf_alphas(alphas, questions)

# Fleis Kappa

In [ ]:
from bench_lib.evaluation import fleiss_kappa

n_raters = 3
q = "is_intolerant"
fk_human_labels = human_labels.query("nraters == @n_raters")
table = pd.crosstab(fk_human_labels["post_id"], fk_human_labels[q]).values
kp, agreements = fleiss_kappa(table, method="fleiss")
ag_counts = pd.Series(agreements).value_counts()
print(kp)
fig = plt.figure(figsize=(4, 3))
plt.bar(ag_counts.index, ag_counts.values, width=0.05)
plt.xlim(-0.1, 1.1)
plt.xlabel("Agreement")
plt.ylabel("Count")
plt.title(f"Agreement for '{q}' with {n_raters} raters. Fleiss Kappa: {kp:.2f}")
plt.tight_layout()
plt.show()

# Post Difficulty

In [ ]:
from matplotlib import pyplot as plt
from bench_lib.evaluation import difficulty_score, get_means

q_mens = {q: get_means(human_labels, question=q) for q in questions}
difficulty = difficulty_score(pd.DataFrame(q_mens))

diff_counts = difficulty.value_counts()
fig = plt.figure()
plt.bar(diff_counts.index, diff_counts.values, width=0.03)
plt.xlabel("Difficulty")
plt.ylabel("Count")
plt.title("Post Difficulty")
plt.tight_layout()

In [ ]:
unique_answers_by_post = (
    human_labels.groupby("post_id")[questions].nunique().max(axis=1)
)
hard_posts = unique_answers_by_post[unique_answers_by_post > 1]
easy_posts = unique_answers_by_post[unique_answers_by_post == 1]
print(len(hard_posts), len(easy_posts))

# How does AI perform on the easy posts?
We assume here the ground truth to be the human labels, which are unique by construction

In [ ]:
from bench_lib.evaluation import load_ai_labels, performance_by_category


human_labels_easy_set = human_labels.query("post_id in @easy_posts.index")[
    ["post_id", *questions]
].drop_duplicates()
human_labels_easy_set

#models = ["gemma-3-4b-it", "gemma-3-12b-it", "gemma-3-27b-it", "MiniCPM-V-2.6"]
folders = [f"gemma-3-27b-it_{n:02d}" for n in range(3)]
all_ai_labels = load_ai_labels(folders, questions, comment_cols)


# performance_by_category(complete_ai_labels, human_labels_easy_set)
all_ai_perfs = (
    all_ai_labels.groupby("model_id")
    .apply(
        performance_by_category,
        ref_wide=human_labels_easy_set,
        questions=questions,
        include_groups=False,
    )
    .reset_index()
    .drop(columns=["level_1"])
)

In [ ]:
all_ai_labels["is_hedonic_entertainment"].unique()

In [ ]:
import plotly.express as px

px.bar(all_ai_perfs, x="variable", y="is_correct", color="model_id", barmode="group")

In [ ]:
from bench_lib.evaluation import join_wides


pd.set_option("display.max_columns", 100)
pd.set_option("display.width", None)

model_id = "MiniCPM-V-2.6"
q_col = "is_hedonic_entertainment"
joined = join_wides(
    all_ai_labels.query("model_id == @model_id"), human_labels_easy_set, questions
)
ai_bad = joined.query("not is_correct and variable == @q_col")
ai_bad.head()


In [ ]:
from bench_lib.evaluation import to_binary_series

from bench_lib.evaluation import plot_question_hists

human_hist_data = {}
for question, means in q_mens.items():
    counts = means.value_counts(normalize=True)
    human_hist_data[question] = (counts.index, counts.values)

ai_hist_data = {}
for question in questions:
    counts = to_binary_series(
        all_ai_labels.query("model_id == @model_id")[question]
    ).value_counts(normalize=True)
    ai_hist_data[question] = (counts.index, counts.values)

plot_question_hists(ai_hist_data, title=model_id)
plot_question_hists(human_hist_data, title="Human")

In [14]:
q_col_comment = q_col + "_comment"
for _, row in all_ai_labels.query(
    "model_id == @model_id and post_id.isin(@ai_bad.post_id)"
)[["post_id", q_col_comment, q_col]].iterrows():
    print(row["post_id"])
    print(row[q_col_comment])
    print(row[q_col])
    print()
